# How are returns and volatility related for energy stocks and the broader market?

## Introduction

**Business Context.** You are an analyst at a large bank focused on natural resource stock investments. You recently conducted an analysis of the following energy stocks and how their trading volume is related to their volatility:

1. Dominion Energy Inc. (Stock Symbol: D)
2. Exelon Corp. (Stock Symbol: EXC)
3. NextEra Energy Inc. (Stock Symbol: NEE)
4. Southern Co. (Stock Symbol: SO)
5. Duke Energy Corp. (Stock Symbol: DUK)

Your boss was quite pleased with your previous analysis, and now wants you to conduct additional analysis so he can figure out how to size potential positions in these stocks... i.e. what percentage of the investment portfolio should be dedicated to each of these stocks. Specifically, he wants you to look at daily returns and volatility for each stock as well as for the broader market (i.e. not just the energy sector).

This is important because high volatility implies higher risk, and your boss would like to know if the potential returns of these high-volatility energy stocks compensate him for the added risk. Additionally, because his performance is measured or benchmarked against the broader market, he wants to understand whether these stocks generally outperform the broader market.

**Business Problem.** Based on the above context, your boss has posed the following question to you: **"What is the relationship between daily volatility and returns for these stocks, and what is the relationship between daily returns for these stocks and the broader stock market?"**

**Analytical Context.** The data you've been given is in the Comma Separated Value (CSV) format, and comprises price and trading volume data for the above stocks. You will proceed by: (1) conducting preliminary cleaning of the data; (2) creating additional features required for our analysis; (3) labelling the data into volatility groups, or regimes, and determining how volatility is related to returns; and finally (4) comparing these returns against those of the broader market.

In [1]:
# Import libraries required for this case
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Preliminary cleaning of the data

Before we can proceed with data analysis and modeling, we first need to determine if the relevant data is adequate to proceed as-is, or if it needs further cleaning. In this case, we have received a Comma Separated Value (CSV) file that includes the following data:

1. **Date:** The day of the year
2. **Open:** The stock opening price of the day
3. **High:** The highest observed stock price of the day
4. **Low:** The lowest observed stock price of the day
5. **Close:** The stock closing price of the day
6. **Adj Close:** The adjusted stock closing price for the day (adjusted for splits and dividends)
7. **Volume:** The volume of the stock traded over the day
8. **Symbol:** The symbol for that particular stock

One very common problem that arises in datasets is missing values. Let's see how to identify whether or not our dataset has this problem, and how to deal with missing values.

In [2]:
# Load and view head of DataFrame
raw_df = pd.read_csv('EnergySectorData.csv')
raw_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D


Let's begin by determining if we have missing values. We can use the ```pandas``` DataFrame method ```isnull()``` to check for ```NaN``` values in ```raw_df``` (i.e. check for null values):

In [3]:
# Check if there are missing values (NaNs)
raw_df.isnull().sum()

Date          0
Open          2
High         14
Low           7
Close         7
Adj Close     7
Volume       22
Symbol        0
dtype: int64

Here we see we have some missing values. Let's instead use the ```mean()``` method to determine what percent of each column we are missing.

In [4]:
raw_df.isnull().mean()

Date         0.000000
Open         0.000319
High         0.002231
Low          0.001116
Close        0.001116
Adj Close    0.001116
Volume       0.003506
Symbol       0.000000
dtype: float64

We see here that we are missing less than 0.5% of the observations in any given column. 



### Exercise 1:
We do not want any missing values in our analysis. Which of the following options is the WORST option for how to proceed in this case?

(a) Fill in any day's missing value with the previous day's value

(b) Replace the missing values by re-gathering the data

(c) Estimate the missing values by interpolating them from the values of other, similar data points

(d) Remove rows from the dataset that contain missing values

**Answer.** (a). We have a couple options that are generally acceptable on how to proceed with missing values:

1. One option is to replace the missing values by re-gathering the data. However, this option is often quite expensive in real man-hours, so we will forgo it for now.

2. Another option is to try to estimate the missing values using some reasonable estimation method, interpolating from other data points. However, this can be complicated and given that such a little amount of our data is missing, we will forgo this option.

3. In practice, a regularly chosen option when only a small amount of data is missing is to just remove the rows that have missing data. This option is generally fine to perform so long as the removed data is an insignificant portion of the data under study. Here we will choose this option as it simplifies the analysis and should not harm any results moving forward.

Answer (a) is problematic because replacing a missing value with the previous day's value doesn't make sense for stocks because stock prices and trading volumes are known to move day-to-day rather than remain unchanged for an extended period of time. Since we will be dealing with daily returns and volatility, this is especially problematic as it defaults any missing day's volatility and return to 0.

Note that these options for cleaning data should be carefully weighed when commencing a new data science study.

Let's clean the missing values by removing them from the data set.

In [5]:
# Remove NaNs from data
# Drop the missing values
progress_df = raw_df.dropna()

## Standardizing dates

We'd like to be able to analyze these stocks together across time. This would be easier is if all the stocks contained non-missing data for the same set of dates. Let's first ascertain if this is the case. One way to do this is to use the ```groupby``` method to group by ```Date```, then use the ```count()``` function to enumerate how many distinct dates we have. Since there are a total of 1259 rows per symbol, there should be a count of 1259 for each symbol.

In [6]:
# How many data rows do we have for each Symbol
progress_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1230,1230,1230,1230,1230,1230,1230
DUK,1249,1249,1249,1249,1249,1249,1249
EXC,1239,1239,1239,1239,1239,1239,1239
NEE,1251,1251,1251,1251,1251,1251,1251
SO,1259,1259,1259,1259,1259,1259,1259


Since most symbols do not have a count of 1259 for their ```Date``` columns, there are clearly some inconsistent values. Some of these duplicates will be missing values (NaNs), so let's enumerate those again first:

In [7]:
# Check if there are missing values (NaNs)
raw_df.isnull().sum()

Date          0
Open          2
High         14
Low           7
Close         7
Adj Close     7
Volume       22
Symbol        0
dtype: int64

As discussed earlier, we can remove missing values as there are not many samples that are missing, and dropping a small number of dates is not expected to significantly impact the analysis:

In [8]:
# Drop the missing values
progress_df = raw_df.dropna().copy()

In [9]:
# How many data rows do we have for each Symbol
progress_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1230,1230,1230,1230,1230,1230,1230
DUK,1249,1249,1249,1249,1249,1249,1249
EXC,1239,1239,1239,1239,1239,1239,1239
NEE,1251,1251,1251,1251,1251,1251,1251
SO,1259,1259,1259,1259,1259,1259,1259


Still, we see that different symbols have different numbers of dates. We'd like all the symbols to have the same set of dates for analysis purposes. Let's create a new ```clean_df``` that corresponds to a DataFrame with the same number of rows for each ```Symbol```, where all symbols share the same set of dates:

In [10]:
set_dates_D = set(progress_df[progress_df['Symbol'] == 'D']['Date'])
set_dates_EXC = set(progress_df[progress_df['Symbol'] == 'EXC']['Date'])
set_dates_NEE = set(progress_df[progress_df['Symbol'] == 'NEE']['Date'])
set_dates_SO = set(progress_df[progress_df['Symbol'] == 'SO']['Date'])
set_dates_DUK = set(progress_df[progress_df['Symbol'] == 'DUK']['Date'])
set_unique_dates = set.intersection(set_dates_D,set_dates_EXC,set_dates_NEE,set_dates_SO,set_dates_DUK)

In [11]:
# Filter new DataFrame for only the dates that are present in every symbol (i.e. the overlapping dates)
clean_df = progress_df[progress_df['Date'].isin(set_unique_dates)].copy()

In [12]:
# Let's take a look
clean_df.groupby('Symbol').count()

,Date,Open,High,Low,Close,Adj Close,Volume
Symbol,,,,,,,
D,1192,1192,1192,1192,1192,1192,1192
DUK,1192,1192,1192,1192,1192,1192,1192
EXC,1192,1192,1192,1192,1192,1192,1192
NEE,1192,1192,1192,1192,1192,1192,1192
SO,1192,1192,1192,1192,1192,1192,1192


Now we see that each symbol has the same number of unique dates. Let's write a quick verification program to ensure the resulting ```clean_df``` does indeed have the same dates for every symbol.

### Exercise 2:
Write code to ensure that each of the symbols share the same set of unique dates. (Hint: use the ```set()``` method.)

**Answer.** One possible solution is given below:

In [13]:
# One possible solution
check_set_dates_D = set(clean_df[clean_df['Symbol'] == 'D']['Date'])
check_set_dates_EXC = set(clean_df[clean_df['Symbol'] == 'EXC']['Date'])
check_set_dates_NEE = set(clean_df[clean_df['Symbol'] == 'NEE']['Date'])
check_set_dates_SO = set(clean_df[clean_df['Symbol'] == 'SO']['Date'])
check_set_dates_DUK = set(clean_df[clean_df['Symbol'] == 'DUK']['Date'])

print(check_set_dates_D == check_set_dates_EXC)
print(check_set_dates_D == check_set_dates_NEE)
print(check_set_dates_D == check_set_dates_SO)
print(check_set_dates_D == check_set_dates_DUK)

True
True
True
True


Now that we've completed the preliminary cleaning of the data, let's move forward with determining the relationships between: (1) stock returns and volatility, and (2) stock returns and broader market returns.

## Adding additional variables required for our analysis

Recall that the original question requires us to investigate both daily stock returns and the volatility of those returns. This means that important measures of interest are:

1. Daily (open to close) stock return
2. Volatility of daily stock return

Why are each of these important?

1. Volatility: Gives insight into amount of price movement in any given day. Volatility is directly related to the level of risk involved in holding the stock.
2. Return: Gives us an idea of the return on investment over a period of time.

Let's calculate these statistics and add them to the DataFrame ```clean_df```:

In [14]:
clean_df['VolStat'] = (clean_df['High'] - clean_df['Low']) / clean_df['Open']
clean_df['Return'] = (clean_df['Close'] / clean_df['Open']) - 1.0
clean_df['Volume_Millions'] = clean_df['Volume'] / 1000000.0 # Volume in Millions (added for convenience)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018


Here we see that we've added three columns to ```clean_df```, namely ```VolStat```, ```Return```, and ```Volume_Millions``` (the last one is just for convenience, as the values in the ```Volume``` column are quite large).

Since we are looking to analyze the relationship between daily volatility and returns, an additional column that makes sense to add is one which says ```True``` when the daily return is positive, and ```False``` when the daily return negative. We can then group days into positive and negative return cohorts and compare the average volatility on those days. We will name this column ```ReturnFlag```.

We can accomplish this using an **anonymous function**; that is, a function that is defined but not named:
```python
lambda arguments: expression
```

The ```lambda``` keyword tells Python that we are using an anonymous function. Next, the ```arguments``` are the name we give to the inputs. It can be ```x```, or ```y```, or whatever the user would like to call it. In this case we will use the name ```row``` for the input argument name as the input will indeed be a row of a DataFrame. The ```expression``` is what is then applied to the ```arguments```; this is the function.

Let's take a look at how we can use anonymous functions to create the ```ReturnFlag``` feature:

In [15]:
clean_df['ReturnFlag'] = clean_df.apply(lambda row: True if row['Return'] > 0 else False, axis=1) # Volume in Millions
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True


Notice that the ```apply()``` method takes in an anonymous function and applies it to the rows of the DataFrame through the use of the second argument ```axis```. ```axis=0``` applies the function to columns, whereas ```axis=1``` applies the function to rows.

So what is happening in the following statement?
```python
clean_df['ReturnFlag'] = clean_df.apply(lambda row: True if row['Return] > 0 else False, axis=1)
```

1. ```pandas``` recogized through the ```apply``` method that it is operating on the ```clean_df``` DataFrame
2. The ```apply``` method takes a function as input that will be applied to the DataFrame ```clean_df```
3. Given the second argument of ```apply``` is ```axis=1``` the input into the anonymous function is a single row
4. For every row, ```row['Return']``` returns the ```Return``` value for that row, and it is subsequently passed through the if statement, returning True if greater than zero and False otherwise
5. The new value is stored in the column ```clean_df['ReturnFlag']```

### Exercise 3:

Using ```apply()``` and ```lambda```, write code to create a new column named ```YYYY``` to ```clean_df```, where the new column is the year of the observation as a string. For instance if the row ```Date``` value is 2014-07-28, then the value in the new column for the year would be '2014'. Recall that you can access the first 4 characters of some string ```my_string``` using ```my_string[:4]```.

**Answer.** One possible solution is given below:

In [16]:
# One possible solution
clean_df['YYYY'] = clean_df.apply(lambda row: row['Date'][:4], axis=1)

Let's move forward with labelling volatility regimes present in the data – these regimes are useful for breaking down the stock return analysis by periods of low, medium, and high volatility. It will allow for more granular analysis than just looking at overall averages without a breakdown.

### Exercise 4:
Using ```apply()``` and ```lambda```, write a script to create a new column in ```clean_df``` named ```AvgDailyPrice``` that calculates an average daily price based on whether or not the daily volume is over 5 million. Set the value of the new column to (Open + High + Low + Close)/4 if the volume is larger than 5 million, or set the value to (High + Low + Close)/3 if the volume is less than or equal to 5 million.

**Answer.** One possible solution is given below:

In [17]:
# One possible solution
clean_df['AvgDailyPrice'] = clean_df.apply(lambda row:(row['Open']+row['High']+row['Low']+row['Close'])/4 if row['Volume'] > 5000000 else (row['High']+row['Low']+row['Close'])/3, axis=1)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000


## Labelling energy sector volatility regimes

Similar to Case 1.2, we'd like to label periods of low and high volatilty in a new column called ```VolLevel``` for each Symbol using some lower and upper bound values. For example, in the case of the Symbol D we'd like to have a new column with value determined by:

```python
if VolStrat > upper_threshold_dict['D']:
    VolLevel = '3_HIGH'
elif VolStrat < lower_threshold_dict['D']:
    VolLevel = '1_LOW'
else:
    VolLevel = '2_MEDIUM'
```

Namely, this labelling should be applied to each row, and the threshold values should correspond to the Symbol for that row. We will group by this column and see if we can find any new insights in the data.

In [18]:
# Determine lower bounds (we choose to use 25th percentile)
lower_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.25).to_dict() # 25th percentile bound
lower_threshold_dict

{'D': 0.010240046986389077,
 'DUK': 0.010018315803797114,
 'EXC': 0.011881680089172456,
 'NEE': 0.010258642787424582,
 'SO': 0.009734019893739423}

In [19]:
# Determine upper bounds (we choose to use 75th percentile)
upper_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.75).to_dict() # 75th percentile bound
upper_threshold_dict

{'D': 0.017960914526108228,
 'DUK': 0.017598380774085175,
 'EXC': 0.021801523265676366,
 'NEE': 0.017680218024250814,
 'SO': 0.016830447068579304}

Again, our goal is to label low, medium, and high volatility periods. Let's define a new column called ```VolLevel``` for each Symbol using some lower and upper bound values. Let's define a custom function that will be applied to each row to achieve this goal.

In [20]:
# Our custom function, input is a row from the agg_df, and the output is a string, either LOW, MEDIUM, or HIGH
def my_custom_row_function(row):
    row_symbol = row['Symbol']    # the Symbol value in the row
    row_volstat = row['VolStat']  # the VolStat value in the row
    
    lower_threshold = lower_threshold_dict[row_symbol] # Dictionary of {string:float}
    upper_threshold = upper_threshold_dict[row_symbol] # Dictionary of {string:float}
    
    # The function decision, return value depending on low, medium, or high volatility
    if row_volstat > upper_threshold:
        return '3_HIGH'
    elif row_volstat < lower_threshold:
        return '1_LOW'
    else:
        return '2_MEDIUM'

Let's now apply the function to each row of the DataFrame ```clean_df``` using a ```lambda``` statement. We store the returned values in the new column ```VolLevel```:

In [21]:
# Apply my_custom_row_function to the Pandas DataFrame, row by row (axis=1)
clean_df['VolLevel'] = clean_df.apply(lambda row: my_custom_row_function(row), axis=1)
clean_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel
0,2014-07-28,69.750000,71.059998,69.750000,70.879997,57.963978,1806400.0,D,0.018781,0.016201,1.8064,True,2014,70.563332,3_HIGH
1,2014-07-29,70.669998,70.980003,69.930000,69.930000,57.187099,2231100.0,D,0.014858,-0.010471,2.2311,False,2014,70.280001,2_MEDIUM
2,2014-07-30,70.000000,70.660004,68.400002,68.970001,56.402020,2588900.0,D,0.032286,-0.014714,2.5889,False,2014,69.343336,3_HIGH
3,2014-07-31,68.629997,68.849998,67.580002,67.639999,55.314388,3266900.0,D,0.018505,-0.014425,3.2669,False,2014,68.023333,3_HIGH
4,2014-08-01,67.330002,68.410004,67.220001,67.589996,55.273487,2601800.0,D,0.017674,0.003861,2.6018,True,2014,67.740000,2_MEDIUM


While the workflow here may seem complex at first, the ability to apply custom functions, group by certain features, and construct summary statistics will prove invaluable as you progress onto more advanced analyses.

### Exercise 5:

Using ```clean_df``` and a ```lambda``` statement within ```apply()```, write a function ```new_custom_function()``` and a script to add a new column to the DataFrame (call it ```EnhancedVolLevel```) that operates in a similar manner as VolLevel but instead gives five volatility level categories using the following logic to determine the label for the volatility level:

```python
if VolStrat > 90th percentile:
    VolLevel = '5_VERY_HIGH'
elif VolStrat > 75th percentile:
    VolLevel = '4_HIGH'
elif VolStrat > 25th percentile:
    VolLevel = '3_MEDIUM'
elif VolStrat > 10th percentile:
    VolLevel = '2_LOW'
else:
    VolLevel = '1_VERY_LOW'
```

Remember each percentile should be calculated by symbol. Use these new labels to see if there are any patterns between volatility levels and the direction of returns. Produce the DataFrame to be able to run the following command:

```python
clean_df.groupby(['Symbol','EnhancedVolLevel'])['ReturnFlag'].mean()
```

**Answer.** One possible solution is given below:

In [22]:
# One possible solution
def new_custom_function(row):
    row_symbol = row['Symbol']    # the Symbol value in the row
    row_volstat = row['VolStat']  # the VolStat value in the row
    
    very_lower_threshold = very_lower_threshold_dict[row_symbol] # Dictionary of {string:float}
    lower_threshold = lower_threshold_dict[row_symbol] # Dictionary of {string:float}
    upper_threshold = upper_threshold_dict[row_symbol] # Dictionary of {string:float}
    very_upper_threshold = very_upper_threshold_dict[row_symbol] # Dictionary of {string:float}
    
    # The function decision, return value depending on very low, low, medium, high, or very high volatility
    if row_volstat > very_upper_threshold:
        return '5_VERY_HIGH'
    elif row_volstat > upper_threshold:
        return '4_HIGH'
    elif row_volstat > lower_threshold:
        return '3_MEDIUM'
    elif row_volstat > very_lower_threshold:
        return '2_LOW'
    else:
        return '1_VERY_LOW'
    
# Find thresholds
very_upper_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.90).to_dict() # 90th percentile bound
upper_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.75).to_dict() # 75th percentile bound
lower_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.25).to_dict() # 25th percentile bound
very_lower_threshold_dict = clean_df.groupby('Symbol')['VolStat'].quantile(0.10).to_dict() # 10th percentile bound

# Calculate and add new column
clean_df['EnhancedVolLevel'] = clean_df.apply(lambda row: new_custom_function(row), axis=1)
print(clean_df.head())

# Result
clean_df.groupby(['Symbol','EnhancedVolLevel'])['ReturnFlag'].mean()

         Date       Open       High        Low      Close  Adj Close  \
0  2014-07-28  69.750000  71.059998  69.750000  70.879997  57.963978   
1  2014-07-29  70.669998  70.980003  69.930000  69.930000  57.187099   
2  2014-07-30  70.000000  70.660004  68.400002  68.970001  56.402020   
3  2014-07-31  68.629997  68.849998  67.580002  67.639999  55.314388   
4  2014-08-01  67.330002  68.410004  67.220001  67.589996  55.273487   

      Volume Symbol   VolStat    Return  Volume_Millions  ReturnFlag  YYYY  \
0  1806400.0      D  0.018781  0.016201           1.8064        True  2014   
1  2231100.0      D  0.014858 -0.010471           2.2311       False  2014   
2  2588900.0      D  0.032286 -0.014714           2.5889       False  2014   
3  3266900.0      D  0.018505 -0.014425           3.2669       False  2014   
4  2601800.0      D  0.017674  0.003861           2.6018        True  2014   

   AvgDailyPrice  VolLevel EnhancedVolLevel  
0      70.563332    3_HIGH           4_HIGH  
1     

Symbol  EnhancedVolLevel
D       1_VERY_LOW          0.508333
        2_LOW               0.573034
        3_MEDIUM            0.558725
        4_HIGH              0.528090
        5_VERY_HIGH         0.408333
DUK     1_VERY_LOW          0.550000
        2_LOW               0.522472
        3_MEDIUM            0.558725
        4_HIGH              0.511236
        5_VERY_HIGH         0.433333
EXC     1_VERY_LOW          0.533333
        2_LOW               0.533708
        3_MEDIUM            0.508389
        4_HIGH              0.533708
        5_VERY_HIGH         0.500000
NEE     1_VERY_LOW          0.566667
        2_LOW               0.550562
        3_MEDIUM            0.567114
        4_HIGH              0.522472
        5_VERY_HIGH         0.500000
SO      1_VERY_LOW          0.516667
        2_LOW               0.511236
        3_MEDIUM            0.530201
        4_HIGH              0.500000
        5_VERY_HIGH         0.516667
Name: ReturnFlag, dtype: float64

We see that volatility and stock returns do not exhibit any strong patterns in terms of the average return direction (positive or negative) for a given volatility regime.

## Comparing stock returns to broader market returns

Now, let's look into the second part of your boss's question: what is the relationship between broader market returns and the returns of these five energy stocks? The S&P 500 Index is a stock index comprised of about 500 large-capitalization public US companies. The index is often used as a representation of the US stock market. If we can determine whether or not there exists a strong relationship between these 5 energy stocks' returns and those of the S&P 500 Index, we can determine if there are signficant idiosyncratic characteristics at play among the energy sector stock returns, or if the returns are solely driven by the broader market.

Market returns for the tradable S&P 500 index ETF (exchange-traded fund) are available in ```SPY.csv``` (the "stock" symbol of the ETF is SPY). Let's load the data and append the daily returns onto the cleaned energy sector data:

In [23]:
# Load file into DataFrame
market_df = pd.read_csv('SPY.csv')

In [24]:
market_df['Symbol'] = 'SPY' # add column for symbol
market_df['Return'] = (market_df['Close'] / market_df['Open']) - 1.0 # calculate return
market_df['VolStat'] = (market_df['High'] - market_df['Low']) / market_df['Open']
market_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,Return,VolStat
0,2014-08-18,196.800003,197.449997,196.690002,197.360001,178.729111,75424000,SPY,0.002846,0.003862
1,2014-08-19,197.839996,198.539993,197.440002,198.389999,179.661896,59135000,SPY,0.002780,0.005560
2,2014-08-20,198.119995,199.160004,198.080002,198.919998,180.141846,72763000,SPY,0.004038,0.005451
3,2014-08-21,199.089996,199.759995,198.929993,199.500000,180.667160,67791000,SPY,0.002059,0.004169
4,2014-08-22,199.339996,199.690002,198.740005,199.190002,180.386368,76107000,SPY,-0.000752,0.004766


We'd like to merge the market returns in ```market_df``` and the energy stock data in ```clean_df```.  This can be accomplished using ```pd.merge()``` - a versatile method to join together DataFrames.

For those of you familiar with SQL, merging and joining DataFrames can be accomplished in much the same way that SQL accomplishes these tasks (if you are not familiar with SQL, don't worry – we will cover it in later cases). In this case, we'd like to use the intersection of dates in ```clean_df``` and ```market_df``` dates as the indices in the merge (i.e. in SQL parlance, we will perform an ```inner``` merge):

In [25]:
# Merge inner (merge market_df onto clean_df using the dates of clean_df as the keys)
merged_df = pd.merge(clean_df, market_df[['Date','Return']], how='inner', on='Date', suffixes=('','_SPY'))

In [26]:
# Check how many dates are in the intersection
merged_df.groupby('Symbol')['Date'].count()

Symbol
D      1178
DUK    1178
EXC    1178
NEE    1178
SO     1178
Name: Date, dtype: int64

In [27]:
merged_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,VolStat,Return,Volume_Millions,ReturnFlag,YYYY,AvgDailyPrice,VolLevel,EnhancedVolLevel,Return_SPY
0,2014-08-18,68.970001,69.250000,68.559998,68.680000,56.164867,1375600.0,D,0.010004,-0.004205,1.3756,False,2014,68.829999,1_LOW,2_LOW,0.002846
1,2014-08-18,31.930000,32.099998,31.730000,31.820000,26.593485,4037400.0,EXC,0.011588,-0.003445,4.0374,False,2014,31.883333,1_LOW,2_LOW,0.002846
2,2014-08-18,96.589996,97.180000,95.889999,96.139999,82.166077,1098200.0,NEE,0.013355,-0.004659,1.0982,False,2014,96.403333,2_MEDIUM,3_MEDIUM,0.002846
3,2014-08-18,43.529999,43.720001,43.279999,43.380001,34.210857,2979800.0,SO,0.010108,-0.003446,2.9798,False,2014,43.460000,2_MEDIUM,3_MEDIUM,0.002846
4,2014-08-18,72.300003,72.650002,71.919998,72.089996,58.080738,1826000.0,DUK,0.010097,-0.002905,1.8260,False,2014,72.219999,2_MEDIUM,3_MEDIUM,0.002846


### Exercise 6:

Using only Symbol D in ```clean_df``` and ```market_df```, use ```pd.merge()``` to determine how many dates are in ```clean_df``` that are not in ```market_df```. Additionally, how many dates are in ```market_df``` that are not in ```clean_df```? (Hint: isnull() may be useful to simplify the solution.)

**Answer.** One possible solution is given below:

In [28]:
# One possible solution

outer_df = pd.merge(clean_df[clean_df['Symbol'] == 'D'][['Date','Return']], market_df[['Date','Return']], how='outer', on='Date', suffixes=('','_SPY'))
print('--- outer merge HEAD ---')
print(outer_df.head())
print('--- outer merge TAIL ---')
print(outer_df.tail())
print('--- outer merge NaN count ---')
outer_df.isnull().sum()

# Answer to: For Symbol D, how many dates are in clean_df that are not in market_df? -> 14
# Answer to: For Symbol D, how many dates are in market_df that are not in clean_df? -> 81

--- outer merge HEAD ---
         Date    Return  Return_SPY
0  2014-07-28  0.016201         NaN
1  2014-07-29 -0.010471         NaN
2  2014-07-30 -0.014714         NaN
3  2014-07-31 -0.014425         NaN
4  2014-08-01  0.003861         NaN
--- outer merge TAIL ---
            Date  Return  Return_SPY
1268  2019-08-12     NaN   -0.006518
1269  2019-08-13     NaN    0.016716
1270  2019-08-14     NaN   -0.014476
1271  2019-08-15     NaN   -0.000807
1272  2019-08-16     NaN    0.008273
--- outer merge NaN count ---


Date           0
Return        81
Return_SPY    14
dtype: int64

### Exercise 7:

Use ```market_df``` and ```clean_df``` to create a new DataFrame ```modified_clean_df``` that is the same as ```clean_df``` but with a new column named ```MeanMonthSPYReturn```. Each value in the new column should be the mean monthly return for SPY for the given month of each row's ```Date```. The output for the first and last rows of the ```modified_clean_df``` DataFrame should produce the following output from head(1) and tail(1):
```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].head(1))
```

                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2014-08-01 D      201408 0.001443

```python
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].tail(1))
```
                 Date       Symbol YYYYMM MeanMonthSPYReturn
                 2019-07-26 DUK    201907 0.000167

**Answer.** One possible solution is given below:

In [29]:
# One possible solution
# Add YYYYMM to market_df and clean_df
market_df['YYYYMM'] = market_df.apply(lambda row: row['Date'][:4] + row['Date'][5:7], axis=1)
clean_df['YYYYMM'] = clean_df.apply(lambda row: row['Date'][:4] + row['Date'][5:7], axis=1)

# Calculate modified_market_df DataFrame for SPY
modified_market_df = market_df.groupby(['Symbol','YYYYMM']).mean().reset_index()
modified_market_df = modified_market_df.rename(columns={'Return':'MeanMonthSPYReturn'}) # rename column

# Inner merge modified_market_df DataFrame and clean_df by YYYYMM
modified_clean_df = pd.merge(clean_df, modified_market_df[['YYYYMM','MeanMonthSPYReturn']], how='inner', on='YYYYMM')

# Print results
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].head(1))
print(modified_clean_df[['Date','Symbol','YYYYMM','MeanMonthSPYReturn']].tail(1))

         Date Symbol  YYYYMM  MeanMonthSPYReturn
0  2014-08-01      D  201408            0.001443
            Date Symbol  YYYYMM  MeanMonthSPYReturn
5939  2019-07-26    DUK  201907            0.000167


## Breaking down returns based on the broader market return

Now we can begin our granular analysis of how broader market returns are related to single stock returns. 

Let's begin by breaking down broader market returns into quantiles. This quantile analysis approach we will take is commonly employed in data analysis to determine how the magnitude of one variable is related to another variable of interest.

We can explore this idea with the ```pd.qcut()``` method. Namely, ```pd.qcut()``` will allow us to cut the market returns by quantiles (we will eventually group by quantiles), and therefore will allow us to calculate summary statistics (such as average return) for each quantile.

Let's first extract the returns using the convenience of the ```pivot()``` method on a DataFrame. Pivoting a DataFrame may be accomplished by specifying:

1. An index to pivot on. In this case we choose ```Date```.
2. Columns that we'd like to have after the pivot. In this case we'd like columns that are the ````Symbol```.
3. The values that each (row,column) pair will show. In this case we'd like to have the ```Return```.

We will pivot ```merged_df``` using these parameter inputs to output a DataFrame where the rows are the Date, each column is the Symbol, and the values are the open-to-close daily return for the given date and symbol.

In [30]:
# Extract returns from merged_df, where we use pivot to simplify the task
return_df = merged_df.pivot(index='Date', columns='Symbol', values='Return')
return_df.head()

Symbol,D,DUK,EXC,NEE,SO
Date,,,,,
2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446
2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741
2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508
2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000
2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195


Let's merge on the broader market returns from the SPY ```market_df``` loaded earlier:

In [31]:
# Let's merge the SPY (broader market) returns by Date onto the return_df DataFrame
full_df = pd.merge(return_df, market_df[['Date','Return']].set_index('Date'), left_index=True, right_index=True)
full_df = full_df.rename(columns={'Return':'MarketReturn'}).reset_index()
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752


Through a few simple lines we've created a DataFrame ```full_df``` where each value is an open-to-close daily return, whether it be for one of the five symbols under study, or for the broader market.

We proceed by utilizing ```pd.qcut()``` with 10 quantiles. In general, the number of quantiles should be chosen based on how granular of a view you require.

In [32]:
# Create 10 quantile categories by the market return
num_quantiles = 10
full_df['market_quantile'] = pd.qcut(full_df['MarketReturn'],num_quantiles,labels=False)
full_df.head()

,Date,D,DUK,EXC,NEE,SO,MarketReturn,market_quantile
0,2014-08-18,-0.004205,-0.002905,-0.003445,-0.004659,-0.003446,0.002846,7
1,2014-08-19,0.013250,0.009838,0.001567,0.008393,0.005741,0.002780,7
2,2014-08-20,0.001438,0.002480,0.000313,0.005349,0.005508,0.004038,7
3,2014-08-21,0.000718,0.003159,0.010022,0.002147,0.000000,0.002059,6
4,2014-08-22,-0.004297,-0.008302,0.006213,-0.003470,-0.003195,-0.000752,3


Now let's group by ```market_quantile``` and calculate the mean return for all the symbols:

In [33]:
# Group by market quantile and calculate the mean return
full_df.groupby('market_quantile').mean()

,D,DUK,EXC,NEE,SO,MarketReturn
market_quantile,,,,,,
0,-0.004553,-0.003282,-0.006711,-0.004723,-0.002890,-0.013363
1,-0.002423,-0.001444,-0.002686,-0.001001,-0.000830,-0.005045
2,-0.001612,-0.000617,-0.001087,-0.000693,-0.000061,-0.002577
3,0.001010,0.000420,-0.001277,0.000728,0.000938,-0.001154
4,0.001189,0.000615,0.000377,0.000903,0.000444,-0.000113
5,-0.001516,-0.001656,-0.000699,-0.000985,-0.001240,0.000887
6,0.001950,0.001233,0.002246,0.002210,0.001040,0.002055
7,0.001804,0.000505,0.001006,0.000831,0.000963,0.003568
8,0.002148,0.002366,0.004323,0.003042,0.002563,0.005787


Each value in the above DataFrame output is a mean of daily returns for a given symbol, where the mean is taken across all dates that correspond to the ```market_quantile``` listed as the index of the output DataFrame. Note that higher quantile numbers indicate higher market returns, while lower quantile numbers indicate lower market returns (in this case, negative market returns).

We see here that the energy stock returns do indeed follow a pattern when the SPY returns are large (higher quantile number) or small (lower quantile number). Namely, stock returns of the individual stocks follow the same pattern as those of the broader market. Hence, the broader market has an effect on the single-stock returns; that is, the magnitude of the broader stock market return is correlated with the magnitude of the return of the individual stocks.

### Exercise 8:

We created quantiles for market returns and subsequently calculated the mean return for each of these quantiles. Perform a similar analysis as above, but instead group by quantiles for the market volatility rather than market returns, and calculate the standard deviation of returns for each market volatility quantile category rather than the mean return.

**Answer.** One possible solution is given below:

In [34]:
# One possible solution
# Merge market volstat onto returns
new_df = pd.merge(return_df, market_df[['Date','VolStat']].set_index('Date'), left_index=True, right_index=True)
new_df = new_df.rename(columns={'VolStat':'MarketVolStat'}).reset_index()

# Add on quantiles
num_quantiles = 10
new_df['market_volstat_quantile'] = pd.qcut(new_df['MarketVolStat'],num_quantiles,labels=False)

# Calcualte mean by quantile
new_df.groupby('market_volstat_quantile').std()

,D,DUK,EXC,NEE,SO,MarketVolStat
market_volstat_quantile,,,,,,
0,0.007394,0.007464,0.009186,0.007310,0.008521,0.000508
1,0.007177,0.007135,0.009354,0.007658,0.006974,0.000262
2,0.008480,0.008084,0.009745,0.007748,0.007407,0.000242
3,0.008298,0.008194,0.010211,0.008106,0.007824,0.000245
4,0.009563,0.008093,0.010409,0.007003,0.008374,0.000301
5,0.009868,0.008642,0.010350,0.008032,0.008836,0.000370
6,0.008900,0.009298,0.013011,0.008751,0.009049,0.000509
7,0.009557,0.009409,0.010587,0.010960,0.008547,0.000674
8,0.010907,0.011419,0.013512,0.010837,0.010354,0.001292


Similar to how stock returns' directions follow those of the broader market, the volatility of stock returns also follow the volatility of returns of the broader market. This warrants further analysis of the root cause of this effect as a future project.

## Conclusions

We've explored stock returns for the five energy sector stocks in terms of their own volatility regimes, and their returns and volatility relative to the broader market. We found that for the stocks under study, there is no strong link between volatility level and the direction of the daily stock return. Moreover, we found that when comparing stocks to the broader market, their returns and volatility levels are amplified when the market returns and volatility levels are high. These findings indicate there is an intrinsic link between returns and volatility, both in the single stock case and the broader market case. This lends a variety of avenues of exploration for follow-up projects.

## Takeaways

In this case, you learned multiple data manipulation tools in ```pandas```, including anonymous functions, grouping, merging, quantile cuttting, and pivoting, while making use of data transformation and aggegation analysis techniques that we've previously learned.

```pandas``` is an increbily versatile package and can significantly increase productivity and deliver exceptional business insights. These techniques should serve as a strong basis for any future data analyses you may conduct.